### Emergency Modeling

In [10]:
import pandas as pd
import os
from datetime import timedelta
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
from sklearn.preprocessing import LabelEncoder
from base.influx_utils import fetch_all_sensor_data
from occupancy_model import (
    prepare_data_for_occupancy_model,
    train_occupancy_model,
    map_sensor_to_room,
    calculate_times_in_each_room
)
from base.minio_utils import save_model_to_minio, load_model_from_minio

In [11]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [23]:
# Fetch sensor data
sensor_data = fetch_all_sensor_data(start_hours=48, interval_hours=24)
sensor_data_df = prepare_data_for_occupancy_model(sensor_data)

In [24]:
sensor_data_df.head(20)

,sensor,bucket,timestamp,value,type,sensor_encoded
351,bathroom_PIR,1_3_6,2025-01-01 20:36:57.317,bathroom,sensor,0
352,bathroom_PIR,1_3_6,2025-01-01 20:37:15.039,bathroom,sensor,0
353,bathroom_PIR,1_3_6,2025-01-01 20:37:22.741,bathroom,sensor,0
354,bathroom_PIR,1_3_6,2025-01-01 20:37:43.750,bathroom,sensor,0
355,bathroom_PIR,1_3_6,2025-01-01 20:38:05.342,bathroom,sensor,0
356,bathroom_PIR,1_3_6,2025-01-01 20:38:23.842,bathroom,sensor,0
357,bathroom_PIR,1_3_6,2025-01-01 20:38:50.978,bathroom,sensor,0
358,bathroom_PIR,1_3_6,2025-01-01 20:39:10.842,bathroom,sensor,0
359,bathroom_PIR,1_3_6,2025-01-01 20:39:45.530,bathroom,sensor,0
360,bathroom_PIR,1_3_6,2025-01-01 20:40:00.413,bathroom,sensor,0


In [25]:
sensor_data_df_aggregated = calculate_times_in_each_room(sensor_data_df) # It will also exclude door events after aggregation
sensor_data_df_aggregated.head(10)

,group_id,room,start_time,end_time,duration,duration_seconds
0,1,bathroom,2025-01-01 20:36:57.317,2025-01-01 20:40:00.413,0 days 00:03:03.096000,183.096
1,2,livingroom,2025-01-01 20:45:46.692,2025-01-01 22:04:35.178,0 days 01:18:48.486000,4728.486
2,3,bathroom,2025-01-01 23:27:11.502,2025-01-02 00:42:30.393,0 days 01:15:18.891000,4518.891
3,4,kitchen,2025-01-02 01:21:35.170,2025-01-02 01:21:44.041,0 days 00:00:08.871000,8.871
4,5,bathroom,2025-01-02 08:18:47.541,2025-01-02 10:37:01.666,0 days 02:18:14.125000,8294.125
5,6,kitchen,2025-01-02 10:52:00.991,2025-01-02 10:52:08.733,0 days 00:00:07.742000,7.742
6,7,livingroom,2025-01-02 10:52:15.954,2025-01-02 10:52:31.072,0 days 00:00:15.118000,15.118
7,8,kitchen,2025-01-02 10:52:31.872,2025-01-02 10:52:31.872,0 days 00:00:00,0.000
8,9,livingroom,2025-01-02 10:52:43.327,2025-01-02 10:52:43.327,0 days 00:00:00,0.000
9,10,kitchen,2025-01-02 10:52:44.006,2025-01-02 10:52:44.006,0 days 00:00:00,0.000


In [26]:
sensor_data_df_aggregated.groupby("room").count()

,group_id,start_time,end_time,duration,duration_seconds
room,,,,,
bathroom,15,15,15,15,15
kitchen,22,22,22,22,22
livingroom,15,15,15,15,15


In [27]:
stats_df = train_occupancy_model(sensor_data_df)
save_model_to_minio(stats_df, "occupancy")

In [28]:
stats1 = load_model_from_minio("occupancy", 1)
stats2 = load_model_from_minio("occupancy", 2)

In [29]:
stats1

,room,mean,std
0,bathroom,906.331600,2345.213762
1,kitchen,213.838955,474.832911
2,livingroom,317.616000,1220.240813


In [30]:
# Second to last model
stats2

,room,mean,std
0,bathroom,208.242158,421.324547
1,kitchen,636.825295,1217.544011
2,livingroom,79.945778,115.666383


### Motion Modeling

In [1]:
from motion_model import train_motion_model
from base.minio_utils import save_model_to_minio, load_model_from_minio

In [2]:
motion_model1 = train_motion_model(start_hours=24*7*6, interval_hours=24*7, time_threshold_seconds=1800)
motion_model2 = train_motion_model(start_hours=24*7*5, interval_hours=24*7, time_threshold_seconds=1800)

In [3]:
motion_model1

,from,to,leave_time,enter_time
0,kitchen,livingroombedarea,2024-11-22 19:36:03.029,2024-11-22 19:36:13.902
1,livingroombedarea,livingroomdoor,2024-11-22 19:36:41.381,2024-11-22 19:36:52.000
2,livingroomdoor,kitchen,2024-11-22 19:36:52.000,2024-11-22 19:36:54.404
3,kitchen,bathroom,2024-11-22 19:39:30.877,2024-11-22 19:39:32.858
4,bathroom,kitchen,2024-11-22 19:39:32.858,2024-11-22 19:39:39.640
...,...,...,...,...
544,livingroomdoor,kitchen,2024-11-28 12:25:31.000,2024-11-28 12:35:55.519
545,kitchen,livingroomdoor,2024-11-28 12:40:24.627,2024-11-28 12:44:23.000
546,livingroomdoor,kitchen,2024-11-28 12:44:23.000,2024-11-28 12:48:28.413
547,kitchen,livingroomdoor,2024-11-28 12:56:54.202,2024-11-28 12:57:06.000


In [4]:
motion_model2

,from,to,leave_time,enter_time
0,livingroomdoor,kitchen,2024-11-29 15:51:17.000,2024-11-29 16:03:53.625
1,kitchen,livingroomdoor,2024-11-29 16:06:44.833,2024-11-29 16:06:54.000
2,livingroomdoor,kitchen,2024-11-29 16:06:54.000,2024-11-29 16:07:01.214
3,kitchen,livingroomdoor,2024-11-29 16:11:49.475,2024-11-29 16:11:52.000
4,livingroomdoor,kitchen,2024-11-29 16:11:52.000,2024-11-29 16:11:56.395
...,...,...,...,...
318,bathroom,kitchen,2024-12-06 12:44:43.474,2024-12-06 12:44:43.797
319,kitchen,bathroom,2024-12-06 12:54:25.683,2024-12-06 12:55:11.787
320,bathroom,kitchen,2024-12-06 12:55:40.744,2024-12-06 12:58:44.665
321,kitchen,bathroom,2024-12-06 13:08:29.198,2024-12-06 13:10:52.828


In [5]:
save_model_to_minio(motion_model2, "motion")
save_model_to_minio(motion_model1, "motion")

In [6]:
old = load_model_from_minio("motion", 2)
new = load_model_from_minio("motion", 1)

In [7]:
old

,from,to,leave_time,enter_time
0,livingroomdoor,kitchen,2024-11-29 15:51:17.000,2024-11-29 16:03:53.625
1,kitchen,livingroomdoor,2024-11-29 16:06:44.833,2024-11-29 16:06:54.000
2,livingroomdoor,kitchen,2024-11-29 16:06:54.000,2024-11-29 16:07:01.214
3,kitchen,livingroomdoor,2024-11-29 16:11:49.475,2024-11-29 16:11:52.000
4,livingroomdoor,kitchen,2024-11-29 16:11:52.000,2024-11-29 16:11:56.395
...,...,...,...,...
318,bathroom,kitchen,2024-12-06 12:44:43.474,2024-12-06 12:44:43.797
319,kitchen,bathroom,2024-12-06 12:54:25.683,2024-12-06 12:55:11.787
320,bathroom,kitchen,2024-12-06 12:55:40.744,2024-12-06 12:58:44.665
321,kitchen,bathroom,2024-12-06 13:08:29.198,2024-12-06 13:10:52.828


In [8]:
new

,from,to,leave_time,enter_time
0,kitchen,livingroombedarea,2024-11-22 19:36:03.029,2024-11-22 19:36:13.902
1,livingroombedarea,livingroomdoor,2024-11-22 19:36:41.381,2024-11-22 19:36:52.000
2,livingroomdoor,kitchen,2024-11-22 19:36:52.000,2024-11-22 19:36:54.404
3,kitchen,bathroom,2024-11-22 19:39:30.877,2024-11-22 19:39:32.858
4,bathroom,kitchen,2024-11-22 19:39:32.858,2024-11-22 19:39:39.640
...,...,...,...,...
544,livingroomdoor,kitchen,2024-11-28 12:25:31.000,2024-11-28 12:35:55.519
545,kitchen,livingroomdoor,2024-11-28 12:40:24.627,2024-11-28 12:44:23.000
546,livingroomdoor,kitchen,2024-11-28 12:44:23.000,2024-11-28 12:48:28.413
547,kitchen,livingroomdoor,2024-11-28 12:56:54.202,2024-11-28 12:57:06.000
